In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from scipy.ndimage.interpolation import shift

In [2]:
# Versões dos pacotes usados no notebook

%reload_ext watermark
%watermark -a "César S. Hooper/ Novembro 2022" --iversions
!python --version

Author: César S. Hooper/ Novembro 2022

pandas    : 1.2.4
matplotlib: 3.3.4
cv2       : 4.5.4-dev
numpy     : 1.19.5

Python 3.8.8


In [9]:
path = 'C:/Users/Cesar Hooper/Desktop/github/bolsa_chile/resumen.csv'

In [22]:
datos = pd.read_excel('micro emini ES 08-07 18_43.xls')

In [23]:
datos.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,07:10,NaN,3914.25,I,I,L,S,-113,84,NaN,NaN,NaN,NaN,NaN,NaN
1,07:11,NaN,3915.25,I,I,L,S,-67,86,NaN,NaN,NaN,NaN,NaN,NaN
2,07:12,NaN,3915.75,I,I,L,S,-56,87,BUY,NaN,NaN,NaN,NaN,NaN
3,07:13,NaN,3915.50,I,I,L,S,-56,86,-0.25,NaN,NaN,NaN,NaN,NaN
4,07:14,NaN,3916.00,I,I,L,S,-45,87,0.25,NaN,NaN,NaN,NaN,NaN


In [24]:
col = [];
for k in range(1, 16):
    col.append('x' + str(k));
    
col = pd.Series(col);    
col

0      x1
1      x2
2      x3
3      x4
4      x5
5      x6
6      x7
7      x8
8      x9
9     x10
10    x11
11    x12
12    x13
13    x14
14    x15
dtype: object

In [25]:
datos.columns = col
datos.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,07:10,NaN,3914.25,I,I,L,S,-113,84,NaN,NaN,NaN,NaN,NaN,NaN
1,07:11,NaN,3915.25,I,I,L,S,-67,86,NaN,NaN,NaN,NaN,NaN,NaN
2,07:12,NaN,3915.75,I,I,L,S,-56,87,BUY,NaN,NaN,NaN,NaN,NaN
3,07:13,NaN,3915.50,I,I,L,S,-56,86,-0.25,NaN,NaN,NaN,NaN,NaN
4,07:14,NaN,3916.00,I,I,L,S,-45,87,0.25,NaN,NaN,NaN,NaN,NaN


In [66]:
dataset = datos[['x1', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x15']]
dataset.head()

,x1,x3,x4,x5,x6,x7,x8,x9,x15
0,07:10,3914.25,I,I,L,S,-113,84,NaN
1,07:11,3915.25,I,I,L,S,-67,86,NaN
2,07:12,3915.75,I,I,L,S,-56,87,NaN
3,07:13,3915.50,I,I,L,S,-56,86,NaN
4,07:14,3916.00,I,I,L,S,-45,87,NaN


In [67]:
colnew = [0 if k == 'buena' else 1 for k in dataset['x15']]       
colnew = pd.Series(colnew);

In [68]:
dataset['target'] = colnew;

<ipython-input-68-9714a45d117f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['target'] = colnew;


In [69]:
dataset = dataset.drop('x15', axis=1);

In [70]:
dataset.head()

,x1,x3,x4,x5,x6,x7,x8,x9,target
0,07:10,3914.25,I,I,L,S,-113,84,1
1,07:11,3915.25,I,I,L,S,-67,86,1
2,07:12,3915.75,I,I,L,S,-56,87,1
3,07:13,3915.50,I,I,L,S,-56,86,1
4,07:14,3916.00,I,I,L,S,-45,87,1


In [80]:
x_hora = dataset['x1'];
dataset = dataset.drop('x1', axis=1);
dataset = pd.get_dummies(dataset);
dataset.head()

,x3,x8,x9,target,x4_I,x4_L,x4_O,x4_S,x5_I,x5_L,x5_S,x6_I,x6_L,x6_O,x6_S,x7_O,x7_S
0,3914.25,-113,84,1,1,0,0,0,1,0,0,0,1,0,0,0,1
1,3915.25,-67,86,1,1,0,0,0,1,0,0,0,1,0,0,0,1
2,3915.75,-56,87,1,1,0,0,0,1,0,0,0,1,0,0,0,1
3,3915.50,-56,86,1,1,0,0,0,1,0,0,0,1,0,0,0,1
4,3916.00,-45,87,1,1,0,0,0,1,0,0,0,1,0,0,0,1
